[View in Colaboratory](https://colab.research.google.com/github/hatu87/tensorflow-sample/blob/master/tf_eager_iris.ipynb)

In [0]:
!pip install -q --upgrade tensorflow

In [0]:
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

In [7]:
print('tensorflow version: {}'.format(tf.__version__))
print('eager execution state: {}'.format(tf.executing_eagerly()))

tensorflow version: 1.8.0
eager execution state: True


In [0]:
train_dataset_url = 'http://download.tensorflow.org/data/iris_training.csv'